In [1]:
from pyignite import Client
from pyignite import GenericObjectMeta, Client
from pyignite.datatypes import String, IntObject, DoubleObject
from pyignite.datatypes.prop_codes import PROP_NAME, PROP_SQL_SCHEMA, PROP_QUERY_ENTITIES
import pandas as pd

In [2]:
CACHE_CONFIG = {
    PROP_NAME: 'SQL_PUBLIC_STUDENT',
    PROP_SQL_SCHEMA: 'PUBLIC',
    PROP_QUERY_ENTITIES: [
        {
            'table_name': 'Student'.upper(),
            'key_field_name': 'sid',
            'key_type_name': 'java.lang.Long',
            'field_name_aliases': [],
            'query_fields': [
                {
                    'name': 'sid',
                    'type_name': 'java.lang.Long',
                    'is_key_field': True,
                    'is_notnull_constraint_field': True,
                },
                {
                    'name': 'name',
                    'type_name': 'java.lang.String',
                },
                {
                    'name': 'login',
                    'type_name': 'java.lang.String',
                },
                {
                    'name': 'age',
                    'type_name': 'java.lang.Integer',
                },
                {
                    'name': 'gpa',
                    'type_name': 'java.math.Double',
                },
            ],
            'query_indexes': [],
            'value_type_name': 'SQL_PUBLIC_STUDENT_TYPE',
            'value_field_name': None,
        },
    ],
}

In [3]:
class Student(
    metaclass=GenericObjectMeta,
    type_name='SQL_PUBLIC_STUDENT_TYPE',
    schema={
        'name': String,
        'login': String,
        'age': IntObject,
        'gpa': DoubleObject,
    }
):
    pass

In [4]:
students = [
    {'name': 'John Doe', 'login': 'john_doe', 'age': 18, 'gpa': 2.5},
    {'name': 'Jane Doe', 'login': 'jane_doe', 'age': 25, 'gpa': 3.5},
    {'name': 'Eva Lu Ator', 'login': 'eva_lisp', 'age': 45, 'gpa': 4.9}
]

In [5]:
client = Client(partition_aware=True)

In [25]:
with client.connect([('127.0.0.1', 10800 + i) for i in range(3)]):
    cache = client.get_or_create_cache(CACHE_CONFIG)
    cache.put_all({i: Student(**st) for i, st in enumerate(students)})
    
            
    with client.sql('select * from STUDENT', include_field_names=True) as cursor:
        row_names = next(cursor)
        rows = list(cursor)
        df = pd.DataFrame(rows, columns=row_names)

df

,SID,NAME,LOGIN,AGE,GPA
0,0,John Doe,john_doe,18,2.5
1,1,Jane Doe,jane_doe,25,3.5
2,2,Eva Lu Ator,eva_lisp,45,4.9
